## "Table 3" extraction from ISS weekly covid-19 reports
https://www.epicentro.iss.it/coronavirus/sars-cov-2-sorveglianza-dati

See example pdf: 
https://www.epicentro.iss.it/coronavirus/bollettino/Bollettino-sorveglianza-integrata-COVID-19_8-settembre-2021.pdf

Requirements: Java 8+, Python 3.6+

In [96]:
from tabula import read_pdf
import pandas as pd
import numpy as np

In [97]:
# input file
in_file = 'foo.pdf' # replace with pdf name

# report data
rep_date = '16/09/2021'

# ...and read the raw table
raw_tb = read_pdf(in_file, pages=17, stream=True) # set page containing table 3
raw_tb[0]

,Unnamed: 0,FASCIA DI,Unnamed: 1,VACCINATI CON,VACCINATI CON.1
0,GRUPPO,NaN,NON VACCINATI,NaN,NaN
1,NaN,ETÀ,NaN,CICLO INCOMPLETO,CICLO COMPLETO
2,NaN,12-39,"7.378.291 (42,3%)",3.129.839 (18%),"6.927.425 (39,7%)"
3,Popolazione,40-59,"4.990.358 (27,1%)","1.422.424 (7,7%)","12.034.678 (65,2%)"
4,(21/08/2021),60-79,"1.847.629 (13,6%)","552.988 (4,1%)","11.172.162 (82,3%)"
5,NaN,80+,"291.252 (6,4%)","105.042 (2,3%)","4.157.813 (91,3%)"
6,NaN,12-39,"64.612 (74,6%)","10.267 (11,9%)","11.719 (13,5%)"
7,Diagnosi di,NaN,NaN,NaN,NaN
8,Sars-CoV-2,40-59,"29.879 (59,6%)","4.232 (8,4%)","15.993 (31,9%)"
9,(06/08/2021-,60-79,"8.631 (40,5%)","1.467 (6,9%)","11.201 (52,6%)"


In [98]:
# keep the last and the third last column
columns_to_keep = [raw_tb[0].columns[-3], raw_tb[0].columns[-1]]

to_exclude = '\((.*)|[^a-z-0-9]|\d+-\d+|\d+\+'
df = raw_tb[0][columns_to_keep].replace(to_exclude, '', regex=True).replace('', np.nan)
df = df.dropna(subset=columns_to_keep, how='all').fillna(0).astype(np.int64)
df.columns = ['Non vaccinati', 'Immunizzati']
df

,Non vaccinati,Immunizzati
2,7378291,6927425
3,4990358,12034678
4,1847629,11172162
5,291252,4157813
6,64612,11719
8,29879,15993
9,8631,11201
10,1768,4862
11,1711,105
12,2459,258


In [99]:
# get data
# sum value by age/condition

step_ = 4                  # values for age and condition (=5) are 4 rows (=20) distant (see foo.pdf)

results = [df[col][i:i+step_].sum() for i in np.arange(0, len(df)-step_+1, step_) for col in df.columns]
results

[14507530, 34292078, 104890, 43775, 6579, 2118, 712, 157, 571, 280]

In [100]:
# read the original general data csv from apalladi's repo
# https://github.com/apalladi/covid_vaccini_monitoraggio/tree/main/dati
date_format = '%d/%m/%Y'
date_parser = lambda x: pd.to_datetime(x, format=date_format)
url = 'https://raw.githubusercontent.com/apalladi/covid_vaccini_monitoraggio/main/dati/dati_ISS_complessivi.csv'
df_0 = pd.read_csv(url, sep=';', parse_dates=['data'], date_parser=date_parser, index_col='data')
df_0

,non vaccinati,vaccinati completo,casi non vaccinati,casi vaccinati,ospedalizzati non vaccinati,ospedalizzati vaccinati,terapia intensiva non vaccinati,terapia intensiva vaccinati,decessi non vaccinati,decessi vaccinati
data,,,,,,,,,,
2021-09-08,14507530,34292078,104890,43775,6579,2118,712,157,571,280
2021-09-01,15656647,32287644,107937,40380,5988,1788,641,127,383,187
2021-08-25,16708830,29628678,104405,33894,5162,1349,500,95,246,92
2021-08-18,17568325,26501452,95261,24978,4052,859,334,61,177,48
2021-08-11,20322716,22345659,80585,18887,3067,627,247,34,143,34
2021-08-04,21289761,19570464,60267,12333,2234,404,169,23,128,34
2021-07-28,22879167,17389604,40729,7277,1619,280,123,16,169,38
2021-07-21,24745853,15384196,26284,3805,1483,224,117,8,250,46
2021-07-14,27065063,13929401,21089,2310,1880,240,147,7,357,68


In [101]:
# add the new row at the top of the df
df_0.loc[pd.to_datetime(rep_date)] = results
df_0.sort_index(ascending=False, inplace=True)
df_0

,non vaccinati,vaccinati completo,casi non vaccinati,casi vaccinati,ospedalizzati non vaccinati,ospedalizzati vaccinati,terapia intensiva non vaccinati,terapia intensiva vaccinati,decessi non vaccinati,decessi vaccinati
data,,,,,,,,,,
2021-09-16,14507530,34292078,104890,43775,6579,2118,712,157,571,280
2021-09-08,14507530,34292078,104890,43775,6579,2118,712,157,571,280
2021-09-01,15656647,32287644,107937,40380,5988,1788,641,127,383,187
2021-08-25,16708830,29628678,104405,33894,5162,1349,500,95,246,92
2021-08-18,17568325,26501452,95261,24978,4052,859,334,61,177,48
2021-08-11,20322716,22345659,80585,18887,3067,627,247,34,143,34
2021-08-04,21289761,19570464,60267,12333,2234,404,169,23,128,34
2021-07-28,22879167,17389604,40729,7277,1619,280,123,16,169,38
2021-07-21,24745853,15384196,26284,3805,1483,224,117,8,250,46


In [102]:
# change data index to match the original format
df_0.index = df_0.index.strftime(date_format)

# save to a csv
df_0.to_csv('dati_ISS_complessivi.csv', encoding='utf-8-sig')

In [103]:
# get data by age
ages = ['12-39', '40-59', '60-79', '80+']
results_ = {age: df[ages.index(age)::step_].stack().values for age in ages}
results_

{'12-39': array([7378291, 6927425,   64612,   11719,    1711,     105,      63,
              3,       7,       0], dtype=int64),
 '40-59': array([ 4990358, 12034678,    29879,    15993,     2459,      258,
             275,       17,       85,       10], dtype=int64),
 '60-79': array([ 1847629, 11172162,     8631,    11201,     1771,      736,
             331,       91,      257,       62], dtype=int64),
 '80+': array([ 291252, 4157813,    1768,    4862,     638,    1019,      43,
             46,     222,     208], dtype=int64)}

In [104]:
# load dict as df
df_1 = pd.DataFrame(results_).T
df_1.columns = df_0.columns
df_1.index.rename('età', inplace=True)
df_1.head()

,non vaccinati,vaccinati completo,casi non vaccinati,casi vaccinati,ospedalizzati non vaccinati,ospedalizzati vaccinati,terapia intensiva non vaccinati,terapia intensiva vaccinati,decessi non vaccinati,decessi vaccinati
età,,,,,,,,,,
12-39,7378291,6927425,64612,11719,1711,105,63,3,7,0
40-59,4990358,12034678,29879,15993,2459,258,275,17,85,10
60-79,1847629,11172162,8631,11201,1771,736,331,91,257,62
80+,291252,4157813,1768,4862,638,1019,43,46,222,208


In [105]:
# save to csv
out_date = rep_date.replace('/', '-')
out_name = f'data_iss_età_{out_date}.csv'
df_1.to_csv(out_name, encoding='utf-8-sig')